# 6. 对抗生成网络
生成模型（Generative Model)  
自动编码器(AutoEncoder)  
变分自动编码器(Variational Autoencoder, VAE)  

## 6.1 Generative Model
生成模型的功能：“生成”的样本和“真实”的样本尽可能相似。  
学习一个概率分布$P_{model}(X)$ 和生成数据。

### 6.1.1 AutoEncoder  


In [1]:
import os

import torch
from torch.autograd import Variable
from torch import nn
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision import transforms as tfs
from torchvision.utils import save_image

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
        nn.Linear(28*28, 128),
        nn.ReLU(True),
        nn.Linear(128, 64),
        nn.ReLU(True),
        nn.Linear(64, 12),
        nn.ReLU(True),
        nn.Linear(12, 3)
        )
        
        self.decoder = nn.Sequential(
        nn.Linear(3, 12),
        nn.ReLU(True),
        nn.Linear(12, 64),
        nn.ReLU(True),
        nn.Linear(64, 128),
        nn.ReLU(True),
        nn.Linear(128, 28*28),
        nn.Tanh())
    
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return encode, decode

In [2]:
im_tfs = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # 标准化
])

train_set = MNIST('./mnist', transform=im_tfs, download=True)
train_data = DataLoader(train_set, batch_size=128, shuffle=True)

In [3]:
net = Autoencoder()
x = Variable(torch.randn(1, 28*28)) # batch size 是 1
encode, _ = net(x)
print(encode.shape)

torch.Size([1, 3])


In [5]:
criterion = nn.MSELoss(reduction='sum')  #size_average=False 修改了。
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)


def to_img(x):
    '''
    定义一个函数将最后的结果转换回图片
    '''
    x = 0.5 * (x + 1.)
    x = x.clamp(0, 1)
    x = x.view(x.shape[0], 1, 28, 28)
    return x

In [7]:
import pixiedust

Pixiedust database opened successfully


In [10]:
#%%pixie_debugger
# 开始训练自动编码器
for e in range(100):
    for i, im in enumerate(train_data):
        im = im.view(im.shape[0], -1)
        im = Variable(im)
        # 前向传播
        _, output = net(im)
        loss = criterion(output, im) / im.shape[0] # 平均
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 20 == 0: # 每 20 次，将生成的图片保存一下
        print('epoch: {}, Loss: {:.4f}'.format(e + 1, loss.data[0]))
        pic = to_img(output.cpu().data)
        if not os.path.exists('./simple_autoencoder'):
            os.mkdir('./simple_autoencoder')
        save_image(pic, './simple_autoencoder/image_{}.png'.format(e + 1))

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]